In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("logistic-regression").getOrCreate()

23/02/07 00:18:02 WARN Utils: Your hostname, imhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.16 instead (on interface en0)
23/02/07 00:18:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/07 00:18:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/07 00:18:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer

# pipeline 데이터의 여러가지 처리 단계를 거칠 때 쓸 수 있음 
# 대표적으로 text data tokenize를 해서 onehotencoing 을 파이프라인을 만들 수 있음 

In [3]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [4]:
training.show()

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



In [5]:
# text split
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [6]:
lr = LogisticRegression(maxIter=30, regParam=0.01)

In [14]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [15]:
model = pipeline.fit(training)

23/02/07 00:24:43 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/02/07 00:24:43 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/02/07 00:24:43 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/07 00:24:43 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [16]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])


In [17]:
prediction = model.transform(test)

In [18]:
prediction.show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[19036,68...|[0.37446592544992...|[0.59253766108921...|       0.0|
|  5|             l m n|           [l, m, n]|(262144,[1303,526...|[2.83313663883221...|[0.94444041965042...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[173558,1...|[-1.1622095936644...|[0.23826602233961...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[68303,19...|[3.75513183310007...|[0.97713755512847...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+

